# 1. 라이브러리 임포트 및 설정


In [11]:
import pandas as pd
import numpy as np
import os
import glob
import re
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# 머신러닝 라이브러리
import lightgbm_0814 as lgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error

# 날짜 관련
import holidays

# 시각화 (필요시)
import matplotlib.pyplot as plt
import seaborn as sns

# 랜덤 시드 고정
np.random.seed(42)

print("=== 곤지암 리조트 최종 수요예측 모델 ===")
print("라이브러리 로드 완료")


=== 곤지암 리조트 최종 수요예측 모델 ===
라이브러리 로드 완료


# 2. 유틸리티 함수 정의


In [12]:
def get_korean_season_info(date):
    """한국 계절 정보 반환"""
    month = date.month
    
    season_info = {
        'spring': month in [3, 4, 5],
        'summer': month in [6, 7, 8], 
        'autumn': month in [9, 10, 11],
        'winter': month in [12, 1, 2],
        'ski_season': month in [12, 1, 2, 3],      # 스키시즌
        'summer_vacation': month in [7, 8],        # 여름휴가철
        'autumn_foliage': month in [10, 11],       # 단풍철
        'cherry_blossom': month in [4, 5],         # 벚꽃철
    }
    
    return season_info

def get_holiday_info(date):
    """한국 공휴일 정보 반환"""
    kr_holidays = holidays.SouthKorea()
    
    holiday_info = {
        'is_holiday': date in kr_holidays,
        'is_weekend': date.weekday() >= 5,
        'is_friday': date.weekday() == 4,  # 금요일 (주말 연휴 시작)
        'is_monday': date.weekday() == 0,  # 월요일 (주말 연휴 끝)
    }
    
    holiday_info['is_long_weekend'] = holiday_info['is_holiday'] or holiday_info['is_weekend']
    
    return holiday_info

def safe_numeric_conversion(df, columns):
    """안전한 숫자형 변환"""
    for col in columns:
        if col in df.columns:
            # 문자열을 숫자로 변환, 변환 불가능한 값은 0으로
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
    return df

print("유틸리티 함수 정의 완료")

유틸리티 함수 정의 완료


# 2. 데이터 로드 및 전처리


In [13]:
# 훈련 데이터 로드
print("데이터 로드 중...")
train_df = pd.read_csv('./train/train.csv')
train_df['영업일자'] = pd.to_datetime(train_df['영업일자'])

print(f"훈련 데이터 크기: {train_df.shape}")
print(f"날짜 범위: {train_df['영업일자'].min()} ~ {train_df['영업일자'].max()}")

# 업장 정보 추출
menu_columns = [col for col in train_df.columns if col != '영업일자']

# 숫자형 데이터로 안전하게 변환
train_df = safe_numeric_conversion(train_df, menu_columns)

restaurants = {}
for col in menu_columns:
    if '_' in col:
        restaurant = col.split('_')[0]
        if restaurant not in restaurants:
            restaurants[restaurant] = []
        restaurants[restaurant].append(col)

print(f"총 {len(restaurants)}개 업장:")
for restaurant, menus in restaurants.items():
    print(f"  - {restaurant}: {len(menus)}개 메뉴")


데이터 로드 중...
훈련 데이터 크기: (102676, 3)
날짜 범위: 2023-01-01 00:00:00 ~ 2024-06-15 00:00:00
총 1개 업장:
  - 영업장명: 1개 메뉴


# 3. 피처 엔지니어링


In [14]:
print("\n고급 피처 생성 중...")

# 기본 날짜 피처
train_df['연도'] = train_df['영업일자'].dt.year
train_df['월'] = train_df['영업일자'].dt.month
train_df['일'] = train_df['영업일자'].dt.day
train_df['요일'] = train_df['영업일자'].dt.dayofweek
train_df['월중순서'] = ((train_df['일'] - 1) // 7) + 1  # 월의 몇 번째 주

# 계절 및 공휴일 피처 생성
season_features = []
holiday_features = []

for idx, row in train_df.iterrows():
    date = row['영업일자']
    
    # 계절 정보
    season_info = get_korean_season_info(date)
    season_features.append(season_info)
    
    # 공휴일 정보  
    holiday_info = get_holiday_info(date)
    holiday_features.append(holiday_info)

# 피처를 DataFrame에 추가
for key in season_features[0].keys():
    train_df[key] = [item[key] for item in season_features]

for key in holiday_features[0].keys():
    train_df[key] = [item[key] for item in holiday_features]

# 업장별 피처 생성
for restaurant, menu_cols in restaurants.items():
    # 해당 컬럼들이 모두 숫자형인지 확인
    valid_cols = []
    for col in menu_cols:
        if col in train_df.columns:
            # 숫자형으로 변환 가능한 컬럼만 사용
            try:
                train_df[col] = pd.to_numeric(train_df[col], errors='coerce').fillna(0)
                valid_cols.append(col)
            except:
                print(f"Warning: {col} 컬럼을 숫자형으로 변환할 수 없습니다.")
    
    if len(valid_cols) > 0:
        # 일일 총 주문량
        train_df[f'{restaurant}_총주문량'] = train_df[valid_cols].sum(axis=1)
        
        # 메뉴 다양성 (주문된 메뉴 개수) - 안전한 방식으로 계산
        train_df[f'{restaurant}_메뉴다양성'] = (train_df[valid_cols] > 0).sum(axis=1)
        
        # 평균 메뉴당 주문량
        diversity_col = f'{restaurant}_메뉴다양성'
        train_df[f'{restaurant}_평균주문량'] = train_df[f'{restaurant}_총주문량'] / train_df[diversity_col].replace(0, 1)
        
        # 업장별 방문객 추정 (총주문량 / 2.5)
        train_df[f'{restaurant}_추정방문객'] = np.maximum(1, train_df[f'{restaurant}_총주문량'] // 2.5)

print("기본 피처 생성 완료")


고급 피처 생성 중...
기본 피처 생성 완료


# 5. 시계열 피처 생성


In [15]:
print("시계열 피처 생성 중...")

# 데이터를 날짜순으로 정렬
train_df = train_df.sort_values('영업일자').reset_index(drop=True)

# 시계열 피처 생성
for restaurant in restaurants.keys():
    total_col = f'{restaurant}_총주문량'
    
    if total_col in train_df.columns:
        # 7일 이동평균
        train_df[f'{restaurant}_7일평균'] = train_df[total_col].rolling(window=7, min_periods=1).mean()
        
        # 전일 대비 증감률
        train_df[f'{restaurant}_전일증감률'] = train_df[total_col].pct_change().fillna(0)
        
        # 지난주 동일 요일 대비
        prev_week = train_df[total_col].shift(7).replace(0, 1).fillna(1)
        train_df[f'{restaurant}_지난주대비'] = train_df[total_col] / prev_week

print("시계열 피처 생성 완료")

시계열 피처 생성 중...
시계열 피처 생성 완료


# 6. 메뉴 분류 및 상관관계 분석


In [16]:
print("\n메뉴 분류 중...")

def classify_menus(restaurants):
    """메뉴를 메인/서브로 분류"""
    menu_classification = {}
    
    for restaurant, menu_cols in restaurants.items():
        main_menus = []
        beverage_menus = []
        side_menus = []
        
        for menu_col in menu_cols:
            menu_name = menu_col.split('_', 1)[1].lower()
            
            # 음료류
            if any(keyword in menu_name for keyword in ['콜라', '스프라이트', '맥주', '소주', '막걸리', '아메리카노', '라떼', '음료', '차', '물']):
                beverage_menus.append(menu_col)
            # 메인 요리 (밥, 면, 고기, 정식 등)
            elif any(keyword in menu_name for keyword in ['밥', '면', '탕', '찌개', '구이', '정식', '불고기', '갈비', '브런치', '플래터', '스테이크', '파스타']):
                main_menus.append(menu_col)
            # 사이드 메뉴
            else:
                side_menus.append(menu_col)
        
        menu_classification[restaurant] = {
            'main': main_menus,
            'beverage': beverage_menus,
            'side': side_menus
        }
        
        print(f"{restaurant}: 메인 {len(main_menus)}, 음료 {len(beverage_menus)}, 사이드 {len(side_menus)}")
    
    return menu_classification

menu_classification = classify_menus(restaurants)


메뉴 분류 중...
영업장명: 메인 0, 음료 0, 사이드 1


# 7. 메뉴 관계 분석


In [17]:
print("\n메뉴 관계 분석 중...")

def analyze_menu_relationships(df, menu_classification):
    """메뉴 간 관계 분석"""
    menu_rules = {}
    
    for restaurant, menus in menu_classification.items():
        rules = {}
        
        # 메인 메뉴 주문 시 음료 주문 확률
        for main_menu in menus['main']:
            if main_menu not in df.columns:
                continue
                
            main_orders = df[df[main_menu] > 0]
            if len(main_orders) == 0:
                continue
                
            beverage_probs = {}
            for beverage in menus['beverage']:
                if beverage not in df.columns:
                    continue
                    
                prob = len(main_orders[main_orders[beverage] > 0]) / len(main_orders)
                if prob > 0.1:  # 10% 이상인 경우만
                    beverage_orders = main_orders[main_orders[beverage] > 0]
                    if len(beverage_orders) > 0:
                        avg_qty = beverage_orders[beverage].mean()
                        beverage_probs[beverage] = {'prob': prob, 'avg_qty': avg_qty}
            
            if beverage_probs:
                rules[main_menu] = beverage_probs
        
        menu_rules[restaurant] = rules
    
    return menu_rules

menu_rules = analyze_menu_relationships(train_df, menu_classification)
print("메뉴 관계 분석 완료")


메뉴 관계 분석 중...
메뉴 관계 분석 완료


# 8. 방문객 예측 모델 생성


In [18]:
print("\n방문객 예측 모델 학습 중...")

def create_visitor_models(df, restaurants):
    """업장별 방문객 예측 모델 생성"""
    visitor_models = {}
    
    # 피처 정의
    visitor_features = ['월', '일', '요일', '월중순서', 'spring', 'summer', 'autumn', 'winter',
                       'ski_season', 'summer_vacation', 'autumn_foliage', 'cherry_blossom',
                       'is_holiday', 'is_weekend', 'is_friday', 'is_monday', 'is_long_weekend']
    
    for restaurant in restaurants.keys():
        target_col = f'{restaurant}_추정방문객'
        
        if target_col not in df.columns:
            continue
            
        # 피처에 과거 정보 추가
        features = visitor_features.copy()
        
        # 시계열 피처 추가 (있는 경우에만)
        if f'{restaurant}_7일평균' in df.columns:
            features.append(f'{restaurant}_7일평균')
        if f'{restaurant}_전일증감률' in df.columns:
            features.append(f'{restaurant}_전일증감률')
        
        # 피처 데이터 준비
        feature_data = []
        for feature in features:
            if feature in df.columns:
                feature_data.append(df[feature].fillna(0))
            else:
                feature_data.append(pd.Series([0] * len(df)))
        
        X = pd.concat(feature_data, axis=1)
        X.columns = features
        y = df[target_col]
        
        # 모델 학습
        try:
            model = lgb.LGBMRegressor(
                n_estimators=100,
                max_depth=6,
                learning_rate=0.08,
                random_state=42,
                verbosity=-1
            )
            
            model.fit(X, y)
            visitor_models[restaurant] = {
                'model': model,
                'features': features
            }
            print(f"{restaurant} 방문객 모델 학습 완료")
        except Exception as e:
            print(f"{restaurant} 방문객 모델 학습 실패: {e}")
    
    return visitor_models

visitor_models = create_visitor_models(train_df, restaurants)


방문객 예측 모델 학습 중...
영업장명 방문객 모델 학습 완료


# 9. 메뉴별 수요 예측 모델 생성


In [19]:
print("\n메뉴별 예측 모델 학습 중...")

def create_menu_models(df, restaurants, menu_classification, visitor_models):
    """메뉴별 수요 예측 모델 생성"""
    menu_models = {}
    
    for restaurant, menu_cols in restaurants.items():
        restaurant_models = {}
        
        for menu_col in menu_cols:
            if menu_col not in df.columns:
                continue
                
            # 기본 피처
            features = ['월', '일', '요일', 'spring', 'summer', 'autumn', 'winter',
                       'ski_season', 'summer_vacation', 'autumn_foliage',
                       'is_holiday', 'is_weekend', 'is_long_weekend']
            
            # 업장별 정보 추가
            visitor_col = f'{restaurant}_추정방문객'
            avg_col = f'{restaurant}_7일평균'
            
            if visitor_col in df.columns:
                features.append(visitor_col)
            if avg_col in df.columns:
                features.append(avg_col)
            
            # 시계열 피처 생성
            lag1_col = f'{menu_col}_lag1'
            lag7_col = f'{menu_col}_lag7'
            
            df[lag1_col] = df[menu_col].shift(1).fillna(0)
            df[lag7_col] = df[menu_col].shift(7).fillna(0)
            features.extend([lag1_col, lag7_col])
            
            # 같은 업장 다른 메뉴들의 영향
            other_menus = [col for col in menu_cols if col != menu_col and col in df.columns]
            if other_menus:
                other_sum_col = f'{menu_col}_other_sum'
                df[other_sum_col] = df[other_menus].sum(axis=1)
                features.append(other_sum_col)
            
            # 메뉴 타입별 특성
            menu_name = menu_col.split('_', 1)[1].lower()
            
            # 계절별 특성 가중치
            if any(keyword in menu_name for keyword in ['냉면', '아이스', '시원한']):
                summer_boost_col = f'{menu_col}_summer_boost'
                df[summer_boost_col] = df['summer'].astype(int) * 2
                features.append(summer_boost_col)
            elif any(keyword in menu_name for keyword in ['찌개', '탕', '따뜻한']):
                winter_boost_col = f'{menu_col}_winter_boost'
                df[winter_boost_col] = df['winter'].astype(int) * 2
                features.append(winter_boost_col)
            
            # 피처 데이터 준비
            feature_data = []
            for feature in features:
                if feature in df.columns:
                    feature_data.append(df[feature].fillna(0))
                else:
                    feature_data.append(pd.Series([0] * len(df)))
            
            X = pd.concat(feature_data, axis=1)
            X.columns = features
            y = df[menu_col]
            
            # 모델 학습 (수요가 있는 메뉴만)
            if y.sum() > 0 and len(X) > 30:
                try:
                    model = lgb.LGBMRegressor(
                        n_estimators=100,
                        max_depth=5,
                        learning_rate=0.08,
                        random_state=42,
                        verbosity=-1
                    )
                    
                    model.fit(X, y)
                    restaurant_models[menu_col] = {
                        'model': model,
                        'features': features
                    }
                except Exception as e:
                    print(f"{menu_col} 모델 학습 실패: {e}")
        
        menu_models[restaurant] = restaurant_models
        print(f"{restaurant}: {len(restaurant_models)}개 메뉴 모델 학습 완료")
    
    return menu_models

menu_models = create_menu_models(train_df, restaurants, menu_classification, visitor_models)

print(f"\n학습 완료:")
print(f"- 방문객 모델: {len(visitor_models)}개")
print(f"- 메뉴 모델: {sum(len(models) for models in menu_models.values())}개")


메뉴별 예측 모델 학습 중...
영업장명: 0개 메뉴 모델 학습 완료

학습 완료:
- 방문객 모델: 1개
- 메뉴 모델: 0개


# 10. 예측 함수 정의


In [20]:
def estimate_visitors_rule_based(restaurant, date_features):
    """규칙 기반 방문객 수 추정"""
    base_visitors = {
        '느티나무 셀프BBQ': 80,
        '담하': 60,
        '라그로타': 40,
        '미라시아': 70,
        '연회장': 30,
        '카페테리아': 100,
        '포레스트릿': 50,
        '화담숲주막': 35,
        '화담숲카페': 45
    }.get(restaurant, 50)
    
    # 주말/공휴일 효과
    if date_features.get('is_long_weekend', False):
        base_visitors *= 1.4
    
    # 계절 효과
    if date_features.get('summer_vacation', False):
        base_visitors *= 1.2
    elif date_features.get('ski_season', False):
        base_visitors *= 1.3
    elif date_features.get('autumn_foliage', False):
        base_visitors *= 1.1
    
    return max(1, int(base_visitors))

def estimate_menu_rule_based(menu_col, visitors, date_features):
    """규칙 기반 메뉴 수요 추정"""
    menu_name = menu_col.split('_', 1)[1].lower()
    
    # 기본 주문 비율
    base_ratio = 0.2
    
    # 메뉴 타입별 조정
    if any(keyword in menu_name for keyword in ['콜라', '스프라이트', '음료']):
        base_ratio = 0.4  # 음료는 높은 주문률
    elif any(keyword in menu_name for keyword in ['밥', '정식', '브런치']):
        base_ratio = 0.3  # 주식류
    elif any(keyword in menu_name for keyword in ['고기', '불고기', '갈비']):
        base_ratio = 0.25  # 고기류
    elif any(keyword in menu_name for keyword in ['면', '파스타']):
        base_ratio = 0.2  # 면류
    
    # 계절 효과
    if date_features.get('summer', False) and any(keyword in menu_name for keyword in ['냉', '아이스']):
        base_ratio *= 1.5
    elif date_features.get('winter', False) and any(keyword in menu_name for keyword in ['찌개', '탕']):
        base_ratio *= 1.3
    
    # 주말 효과
    if date_features.get('is_weekend', False):
        base_ratio *= 1.2
    
    predicted_quantity = visitors * base_ratio
    return max(0, predicted_quantity)

print("예측 함수 정의 완료")

예측 함수 정의 완료


# 11. 테스트 데이터 예측 실행


In [21]:
print("\n테스트 데이터 예측 중...")

# 테스트 파일들 로드
test_files = glob.glob('./test/TEST_*.csv')
all_predictions = []

print(f"발견된 테스트 파일: {len(test_files)}개")

for test_file in sorted(test_files):
    print(f"처리 중: {os.path.basename(test_file)}")
    
    # 파일명에서 테스트 번호 추출
    test_match = re.search(r'TEST_(\d+)', test_file)
    if not test_match:
        continue
        
    test_num = test_match.group(1)
    
    # 7일간 예측
    for day in range(1, 8):
        pred_date_str = f"TEST_{test_num}+{day}일"
        
        # 가상 날짜 생성 (예측을 위한)
        base_date = datetime(2025, 1, 1) + timedelta(days=int(test_num)*7 + day - 1)
        
        # 날짜 피처 생성
        date_features = {
            '월': base_date.month,
            '일': base_date.day,
            '요일': base_date.weekday(),
            '월중순서': ((base_date.day - 1) // 7) + 1
        }
        
        # 계절/공휴일 피처
        season_info = get_korean_season_info(base_date)
        holiday_info = get_holiday_info(base_date)
        
        date_features.update(season_info)
        date_features.update(holiday_info)
        
        # 업장별 예측
        for restaurant, menu_cols in restaurants.items():
            # 1단계: 방문객 수 예측
            if restaurant in visitor_models:
                visitor_model = visitor_models[restaurant]
                
                # 방문객 예측용 피처 벡터 생성
                visitor_features_vector = []
                for feature in visitor_model['features']:
                    if feature in date_features:
                        visitor_features_vector.append(int(date_features[feature]) if isinstance(date_features[feature], bool) else date_features[feature])
                    else:
                        visitor_features_vector.append(0)  # 시계열 피처는 0으로
                
                try:
                    predicted_visitors = max(1, visitor_model['model'].predict([visitor_features_vector])[0])
                except Exception as e:
                    predicted_visitors = estimate_visitors_rule_based(restaurant, date_features)
            else:
                predicted_visitors = estimate_visitors_rule_based(restaurant, date_features)
            
            # 2단계: 메뉴별 수요 예측
            for menu_col in menu_cols:
                if restaurant in menu_models and menu_col in menu_models[restaurant]:
                    menu_model_info = menu_models[restaurant][menu_col]
                    
                    # 메뉴 예측용 피처 벡터 생성
                    menu_features_vector = []
                    for feature in menu_model_info['features']:
                        if feature in date_features:
                            feature_value = date_features[feature]
                            menu_features_vector.append(int(feature_value) if isinstance(feature_value, bool) else feature_value)
                        elif f'{restaurant}_추정방문객' in feature:
                            menu_features_vector.append(predicted_visitors)
                        else:
                            menu_features_vector.append(0)
                    
                    try:
                        predicted_quantity = max(0, menu_model_info['model'].predict([menu_features_vector])[0])
                    except Exception as e:
                        predicted_quantity = estimate_menu_rule_based(menu_col, predicted_visitors, date_features)
                else:
                    predicted_quantity = estimate_menu_rule_based(menu_col, predicted_visitors, date_features)
                
                all_predictions.append({
                    '영업일자': pred_date_str,
                    '영업장명_메뉴명': menu_col,
                    '매출수량': predicted_quantity
                })

predictions_df = pd.DataFrame(all_predictions)
print(f"예측 완료: {len(predictions_df)} 건")


테스트 데이터 예측 중...
발견된 테스트 파일: 10개
처리 중: TEST_00.csv
처리 중: TEST_01.csv
처리 중: TEST_02.csv
처리 중: TEST_03.csv
처리 중: TEST_04.csv
처리 중: TEST_05.csv
처리 중: TEST_06.csv
처리 중: TEST_07.csv
처리 중: TEST_08.csv
처리 중: TEST_09.csv
예측 완료: 70 건


# 12. 제출 파일 생성


In [22]:
print("\n제출 파일 생성 중...")

# 예측 결과를 딕셔너리로 변환
pred_dict = dict(zip(
    zip(predictions_df['영업일자'], predictions_df['영업장명_메뉴명']),
    predictions_df['매출수량']
))

# 샘플 서브미션 로드 또는 생성
try:
    sample_submission = pd.read_csv('./sample_submission.csv')
    print("샘플 서브미션 파일 로드 완료")
except:
    # 샘플 파일이 없으면 생성
    print("샘플 서브미션 파일 생성 중...")
    all_dates = sorted(predictions_df['영업일자'].unique())
    all_menus = sorted(predictions_df['영업장명_메뉴명'].unique())
    
    rows = []
    for date in all_dates:
        row = {'영업일자': date}
        for menu in all_menus:
            row[menu] = 0
        rows.append(row)
    
    sample_submission = pd.DataFrame(rows)

# 예측값 적용
final_submission = sample_submission.copy()

for row_idx in final_submission.index:
    date = final_submission.loc[row_idx, '영업일자']
    for col in final_submission.columns[1:]:
        if (date, col) in pred_dict:
            final_submission.loc[row_idx, col] = pred_dict[(date, col)]
        else:
            final_submission.loc[row_idx, col] = 0

# 결과 저장
final_submission.to_csv('final_enhanced_submission.csv', index=False, encoding='utf-8-sig')

print(f"\n=== 예측 완료 ===")
print(f"총 예측 건수: {len(predictions_df):,}")
print(f"제출 파일: final_enhanced_submission.csv")


제출 파일 생성 중...
샘플 서브미션 파일 생성 중...

=== 예측 완료 ===
총 예측 건수: 70
제출 파일: final_enhanced_submission.csv


# 13. 결과 분석


In [23]:
print(f"\n=== 예측 통계 ===")
print(f"총 예측 수량: {predictions_df['매출수량'].sum():,.0f}")
print(f"평균 예측 수량: {predictions_df['매출수량'].mean():.2f}")
print(f"최대 예측 수량: {predictions_df['매출수량'].max():.0f}")
print(f"제로 예측 비율: {(predictions_df['매출수량'] == 0).mean()*100:.1f}%")

# 업장별 예측 통계
print(f"\n=== 업장별 예측 통계 ===")
for restaurant in restaurants.keys():
    restaurant_preds = predictions_df[predictions_df['영업장명_메뉴명'].str.startswith(restaurant)]
    if len(restaurant_preds) > 0:
        total_pred = restaurant_preds['매출수량'].sum()
        avg_pred = restaurant_preds['매출수량'].mean()
        print(f"{restaurant}: 총 {total_pred:,.0f}, 평균 {avg_pred:.1f}")

print(f"\n=== 모든 처리 완료 ===")


=== 예측 통계 ===
총 예측 수량: 15
평균 예측 수량: 0.21
최대 예측 수량: 0
제로 예측 비율: 0.0%

=== 업장별 예측 통계 ===
영업장명: 총 15, 평균 0.2

=== 모든 처리 완료 ===
